# Bespokefit workshop live session

## Learning objectives

By the end of this workshop you will be able to:

- 1. Build, configure and save a general bespokefit optimisation workflow
- 2. Build molecule specific optimizaion schemas from the general workflow
- 3. Load QCArchive computed data using OpenFF-QCSubmit as a fitting reference
- 4. Optimize some bespoke torsion parameters
- 5. analysis the results from ForceBalance 
- 6. Generate refernce data locally using xtb on the fly

In [1]:
from openff.qcsubmit.results import TorsionDriveResultCollection
from openff.bespokefit.workflows import BespokeWorkflowFactory
from openff.bespokefit.schema.optimizers import ForceBalanceSchema
from openff.bespokefit.schema.targets import TorsionProfileTargetSchema
from openff.bespokefit.bespoke import Executor
from openff.bespokefit.schema.data import BespokeQCData
from openff.qcsubmit.common_structures import QCSpec
from openff.toolkit.topology import Molecule
from qcportal import FractalClient
from pprint import pprint

## 1. Building the general workflow

Bespokefit aims to provide a reproducable parameter optimization workflow for SMIRNOFF based force fields. As such normal bespokefit execution starts with a general fitting workflow. This captures every process in the workflow along with any ajustable settings such as how the reference data should be generated. Lets start with a basic workflow which should be ready to use for torsion fitting.

In [2]:
# start with an empty workflow 
workflow = BespokeWorkflowFactory(fragmentation_engine=None, parameter_settings=[], target_smirks=[], target_templates=[])
pprint(workflow.dict())

{'expand_torsion_terms': True,
 'fragmentation_engine': None,
 'generate_bespoke_terms': True,
 'initial_force_field': 'openff_unconstrained-1.3.0.offxml',
 'optimizer': {'adaptive_damping': 1.0,
               'adaptive_factor': 0.2,
               'eigenvalue_lower_bound': 0.01,
               'error_tolerance': 1.0,
               'extras': {},
               'finite_difference_h': 0.01,
               'gradient_convergence_threshold': 0.01,
               'initial_trust_radius': -0.25,
               'job_type': 'optimize',
               'max_iterations': 10,
               'minimum_trust_radius': 0.05,
               'n_criteria': 2,
               'normalize_weights': False,
               'objective_convergence_threshold': 0.01,
               'penalty_additive': 1.0,
               'penalty_type': 'L1',
               'step_convergence_threshold': 0.01,
               'type': 'ForceBalance'},
 'parameter_settings': [],
 'target_smirks': [],
 'target_templates': []}


## 2. Build molecule specific schema



In [3]:
# load the molecules
mol = Molecule.from_file("data/bace.sdf")

In [4]:
mol[5]

NGLWidget()

In [5]:
# load the default factory 
workflow = BespokeWorkflowFactory()

In [6]:
# process all molecules
schema = workflow.optimization_schemas_from_molecules(mol[5])

Building Fitting Schema : 100%|███████████████████| 1/1 [00:04<00:00,  4.76s/it]


In [7]:
# pull out the new smirks and show how they transfer between the parent and the fragment

In [8]:
# look into the data in the fitting schema
schema[0].targets[0].reference_data.dict(exclude={"tasks"})

{'type': 'bespoke',
 'qc_spec': {'method': 'B3LYP-D3BJ',
  'basis': 'DZVP',
  'program': 'psi4',
  'spec_name': 'default',
  'spec_description': 'Standard OpenFF optimization quantum chemistry specification.',
  'store_wavefunction': 'none',
  'implicit_solvent': None,
  'keywords': None},
 'target_conformers': 4}

## 3. Loading data from QCArchive

In [9]:
client = FractalClient()

In [10]:
client.list_collections("torsiondrivedataset")

tagline
collection          name                                                                                                 
TorsionDriveDataset Fragment Stability Benchmark                                                                     None
                    Fragmenter paper                                                                                 None
                    OpenFF Amide Torsion Set v1.0                       Amides, thioamides and amidines diversely func...
                    OpenFF Aniline 2D Impropers v1.0                    Substituted aniline derivatives with various e...
                    OpenFF DANCE 1 eMolecules t142 v1.0                                                              None
                    OpenFF Fragmenter Validation 1.0                                                                 None
                    OpenFF Full TorsionDrive Benchmark 1                                                             None
                    OpenFF Gen 2 Torsion Set 1 Roche                                                                 None
                    OpenFF Gen 2 Torsion Set 1 Roche 2                                                               None
                    OpenFF Gen 2 Torsion Set 2 Coverage                                                              None
                    OpenFF Gen 2 Torsion Set 2 Coverage 2                                                            None
                    OpenFF Gen 2 Torsion Set 3 Pfizer Discrepancy                                                    None
                    OpenFF Gen 2 Torsion Set 3 Pfizer Discrepancy 2                                                  None
                    OpenFF Gen 2 Torsion Set 4 eMolecules Discrepancy                                                None
                    OpenFF Gen 2 Torsion Set 4 eMolecules Discrepan...                                               None
                    OpenFF Gen 2 Torsion Set 5 Bayer                                                                 None
                    OpenFF Gen 2 Torsion Set 5 Bayer 2                                                               None
                    OpenFF Gen 2 Torsion Set 6 Supplemental                                                          None
                    OpenFF Gen 2 Torsion Set 6 Supplemental 2                                                        None
                    OpenFF Gen3 Torsion Set v1.0                                             OpenFF Gen3 Torsion Set v1.0
                    OpenFF Group1 Torsions                                                                           None
                    OpenFF Group1 Torsions 2                                                                         None
                    OpenFF Group1 Torsions 3                                                                         None
                    OpenFF Primary Benchmark 1 Torsion Set                                                           None
                    OpenFF Primary Benchmark 2 Torsion Set                                                           None
                    OpenFF Primary TorsionDrive Benchmark 1                                                          None
                    OpenFF Protein Fragments TorsionDrives v1.0         Torsiondrives for selected dihedrals of variou...
                    OpenFF Rowley Biaryl v1.0                                          Torsiondrives of biaryl molecules.
                    OpenFF Substituted Phenyl Set 1                                                                  None
                    OpenFF Substituted Phenyl Set 1 v2.0                Torsiondrives for selected dihedrals of variou...
                    OpenFF Theory Benchmarking Set B3LYP-D3BJ 6-31+Gss  Torsiondrives for benchmarking B3LYP-D3BJ/6-31...
                    OpenFF Theory Benchmarking Set B3LYP-D3BJ DZVP ...     Torsiondrives

In [11]:
# create a result from the dataset we know our molecule is in
result = TorsionDriveResultCollection.from_server(client, "OpenFF-benchmark-ligand-fragments-v1.0", "default")

In [12]:
# check how many results we have
result.n_molecules

368

In [13]:
result.n_results

481

So we have 368 unique molecules and 481 torsiondrives, with some molecules have multipule torsion driven. All scans are 1D however.

In [14]:
records_and_molecules = result.to_records()

In [15]:
# show what we have pulled down
record, torsion_molecule = records_and_molecules[0]

In [16]:
pprint(record.dict())

{'created_on': datetime.datetime(2020, 8, 11, 8, 41, 29, 18631),
 'error': None,
 'extras': {},
 'final_energy_dict': {'[-105]': -3640.4638158997955,
                       '[-120]': -3640.4647679341156,
                       '[-135]': -3640.465163735234,
                       '[-150]': -3640.4642715928676,
                       '[-15]': -3640.4636147417914,
                       '[-165]': -3640.4624015613344,
                       '[-30]': -3640.4652715028706,
                       '[-45]': -3640.465825944451,
                       '[-60]': -3640.4651696332535,
                       '[-75]': -3640.464054289657,
                       '[-90]': -3640.4633727497303,
                       '[0]': -3640.462376071303,
                       '[105]': -3640.463816209257,
                       '[120]': -3640.4647678395318,
                       '[135]': -3640.4651635049663,
                       '[150]': -3640.464272242762,
                       '[15]': -3640.4636146794787,
       

In [17]:
torsion_molecule

NGLWidget(max_frame=23)

In [18]:
for molecule in schema:
    molecule.update_with_results(records_and_molecules)

In [19]:
for molecule in schema:
    print(molecule.ready_for_fitting)

True


## 4. Optimize a bespoke torsion parameter

In [21]:
executor = Executor()

In [ ]:
executor.execute(schema[0])

  0%|          | 0/3 [00:00<?, ?it/s]

making new fb folders in bespoke_task_0
making forcebalance file system in  bespoke_task_0
generating target directory for torsion-21540558
Note: Failed to import the optional openff.evaluator package. 
Note: Failed to import the optional openff.recharge package.


100%|██████████| 3/3 [00:00<00:00,  4.41it/s]
